In [ ]:
import scanpy as sc
import nibabel as nib
import os
import zoom.prepare as pp
import zoom.sc_tool as sct

os.chdir("/slurm/home/yrd/liaolab/nieshuyang/AHBA_sc")

In [ ]:
# Prepare and preprocess SBPs
SBPs = ["UTG","SVG","MEGtheta","MOR","KOR","CMRO2","T1T2","deltaMT"] # All the SBPs can be fetched via neuromaps
# Assign SBPs to parcellations and perform spatial permutation test
for SBP in SBPs:
    SBP_parc, SBP_perm_parc = pp.process_SBP(
        SBP=f"=/HCPMMP/{SBP}/lh.{SBP}.fsLR.32k.func.gii",
        parcellation="/HCPMMP/lh.HCPMMP1.fsLR.32k.label.gii",
        atlas="fsLR", density="32k", hemi="L", n_perm=1000, seed=123
    )
    SBP_parc.to_csv(f"/HCPMMP/{SBP}/{SBP}_HCPMMP.csv")
    SBP_perm_parc.to_csv(f"/HCPMMP/{SBP}/{SBP}_perm_HCPMMP.csv")

In [ ]:
# Optimal AHBA preprocessing pipeline
path = '/AHBA_donor_parc/genes/parcellations'
fname = 'HCPMMP1_acpc_uncorr.nii'
donors = ['9861', '10021', '12876', '14380', '15496', '15697']
expr_HCPMMP, DS_HCPMMP = pp.abagen_ctx(
    atlas={donor: nib.load(f"{path}/S0{i+1}_H0351/{fname}") for i,donor in enumerate(donors)},
    atlas_info="/HCPMMP/HCPMMP1.0_info.csv",
    data_dir="/AHBA/microarray",
    donors_threshold=3, gene_stability_threshold=0.5
)
expr_HCPMMP.to_csv("/HCPMMP/expression_HCPMMP.csv")
DS_HCPMMP.to_csv("/HCPMMP/DS_HCPMMP.csv")


We applied Seurat for the preprocessing and integration of scRNA-seq dataset, see sketch_integration.R. Since we need to further work with Python, count matrix, normalized gene expression matrix, metadata, cell embeddings and UMAP coordinates are extracted from the SeuratObject adult_ctx.rds to construct AnnData object adult_ctx.h5ad with the same information.

In [ ]:
# Gene specificity score normalization
adata = sc.read_h5ad("/scRNA/adult_ctx.h5ad")
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=50)
adata = sct.compute_gss(adata, n_jobs=-1)
adata.write("/scRNA/adult_ctx_gss.h5ad")